In [ ]:
#pip install snowflake-connector-python
#pip install snowflake-sqlalchemy

#install those if not installed yet

In [6]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from google.cloud import storage
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine
from snowflake.connector import connect
from google.cloud import storage
from snowflake.connector import connect
from sqlalchemy import create_engine
from sqlalchemy.dialects import registry


In [7]:
# Connect to snowflake 
# Read credentials from config file
with open('config_dw.json', 'r') as f:
    config = json.load(f)

snowflake_config = config['snowflake']

# Your Snowflake database credentials
account_name = snowflake_config['account_name']
user = snowflake_config['user']
password = snowflake_config['password']
database = snowflake_config['database']
schema = snowflake_config['schema']

# Establish a connection to Snowflake
conn = connect(
    user=user,
    password=password,
    account=account_name,
    database=database,
    schema=schema
)

registry.register('snowflake', 'snowflake.sqlalchemy', 'dialect')

# Create a SQLAlchemy engine for Snowflake
snowflake_engine = create_engine(f'snowflake://{user}:{password}@{account_name}/{database}/{schema}')
connection = snowflake_engine.connect()
results = connection.execute('select current_version()').fetchone()
print(results[0])

8.16.0


In [8]:
!curl ipecho.net/plain

2600:387:f:7810::c

In [9]:
# Recreate the iterator blob_list
blob_list = bucket.list_blobs()

# Initialize a counter for the number of processed files
file_count = 0

# Iterate over each blob
for blob in blob_list:
    blob_name = blob.name
    print(f"Processing blob: {blob_name}")

    # Check if the blob is a file (not a directory) 
    if '.' in blob_name:
        # Check if the file is in the "Sales/NY" directory
        if blob_name.startswith("SALES/NY/") or blob_name == "SALES/NY":
            # Increment the file count
            file_count += 1
            
            # Check if we have processed 50 files
            if file_count > 50:
                print("Reached 50 files, stopping processing.")
                break  # Stop processing further files
                
            # Get the blob
            blob = bucket.blob(blob_name)

            # Download the blob content
            blob_content = blob.download_as_string()

            # Convert blob content to DataFrame
            df = pd.read_csv(StringIO(blob_content.decode('utf-8')))

            # Display the shape of the DataFrame
            print(f"Shape of {blob_name}: {df.shape}")

            # Store the DataFrame into Snowflake
            table_name = blob_name.split('/')[-1].split('.')[0]  # Extract table name from blob name
            df.to_sql(table_name, con=snowflake_engine, if_exists='append', index=False)
            print(f"DataFrame stored in Snowflake table: {table_name}")
        else:
            print("Blob is not in the 'Sales/NY' directory, skipping...")
    else:
        print("Blob is not a file, skipping...")



NameError: name 'bucket' is not defined

In [5]:
# Recreate the iterator blob_list
blob_list = bucket.list_blobs()

# Iterate over each blob
for blob in blob_list:
    blob_name = blob.name
    print(f"Processing blob: {blob_name}")

    #CHANGE THIS CODE TO UPDATE ONLT THE NEEDED FILES (DO NOT UPDATE ALL AGAIN SINCE WE ALREADY HAVE SOME IN SNOWFALKE)
    # Check if the blob is a file (not a directory) 
    if '.' in blob_name in blob_name:
        # Get the blob
        blob = bucket.blob(blob_name)

        # Download the blob content
        blob_content = blob.download_as_string()

        # Convert blob content to DataFrame
        df = pd.read_csv(StringIO(blob_content.decode('utf-8')))

        # Display the shape of the DataFrame
        print(f"Shape of {blob_name}: {df.shape}")

        # Store the DataFrame into Snowflake
        table_name = blob_name.split('.')[0]  # Extract table name from blob name
        df.to_sql(table_name, con=snowflake_engine, if_exists='append', index=False)
        print(f"DataFrame stored in Snowflake table: {table_name}")
    else:
        print("Blob is not a file, skipping...")

Processing blob: 0.json
Blob is not a file, skipping...
Processing blob: MIT_livingwage_calculation.csv
Blob is not a file, skipping...
Processing blob: MIT_typicalannualsalaries.csv
Blob is not a file, skipping...
Processing blob: MIT_typicalexpenses.csv
Blob is not a file, skipping...
Processing blob: NYS crime data.csv
Blob is not a file, skipping...
Processing blob: SALES/NY/0.json
Blob is not a file, skipping...
Processing blob: SALES/NY/1000.json
Blob is not a file, skipping...
Processing blob: SALES/NY/10000.json
Blob is not a file, skipping...
Processing blob: SALES/NY/100000.json
Blob is not a file, skipping...
Processing blob: SALES/NY/101000.json
Blob is not a file, skipping...
Processing blob: SALES/NY/102000.json
Blob is not a file, skipping...
Processing blob: SALES/NY/103000.json
Blob is not a file, skipping...
Processing blob: SALES/NY/104000.json
Blob is not a file, skipping...
Processing blob: SALES/NY/105000.json
Blob is not a file, skipping...
Processing blob: SALES

/var/folders/y5/y4jkcjwx1zsdr2dqg2zg998r0000gn/T/ipykernel_1827/1925320818.py:25: UserWarning: The provided table name 'SCHOOL_DATA' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=snowflake_engine, if_exists='append', index=False)


DataFrame stored in Snowflake table: SCHOOL_DATA


In [7]:
# Recreate the iterator blob_list
blob_list = bucket.list_blobs()

# Initialize a counter for the number of processed files
file_count = 0

# Flag to indicate if we have encountered the starting point
start_processing = False

# Iterate over each blob
for blob in blob_list:
    blob_name = blob.name
    print(f"Processing blob: {blob_name}")

    # Check if the blob is a file (not a directory) 
    if '.' in blob_name:
        # Check if the file is in the "Sales/NY" directory
        if blob_name.startswith("SALES/NY/") or blob_name == "SALES/NY":
            # Increment the file count
            file_count += 1

            # Check if we have reached the starting point
            if not start_processing:
                if file_count == 13:
                    start_processing = True
                else:
                    continue  # Skip until we reach the starting point

            # Check if we have processed 50 files
            if file_count > 50:
                print("Reached 50 files, stopping processing.")
                break  # Stop processing further files

            # Get the blob
            blob = bucket.blob(blob_name)

            # Download the blob content
            blob_content = blob.download_as_string()

            # Convert blob content to DataFrame
            df = pd.read_csv(StringIO(blob_content.decode('utf-8')))

            # Display the shape of the DataFrame
            print(f"Shape of {blob_name}: {df.shape}")

            # Store the DataFrame into Snowflake
            table_name = blob_name.split('/')[-1].split('.')[0]  # Extract table name from blob name
            df.to_sql(table_name, con=snowflake_engine, if_exists='append', index=False)
            print(f"DataFrame stored in Snowflake table: {table_name}")
        else:
            print("Blob is not in the 'Sales/NY' directory, skipping...")
    else:
        print("Blob is not a file, skipping...")


Processing blob: 0.json
Blob is not in the 'Sales/NY' directory, skipping...
Processing blob: MIT_livingwage_calculation.csv
Blob is not in the 'Sales/NY' directory, skipping...
Processing blob: MIT_typicalannualsalaries.csv
Blob is not in the 'Sales/NY' directory, skipping...
Processing blob: MIT_typicalexpenses.csv
Blob is not in the 'Sales/NY' directory, skipping...
Processing blob: NYS crime data.csv
Blob is not in the 'Sales/NY' directory, skipping...
Processing blob: SALES/NY/0.json
Processing blob: SALES/NY/1000.json
Processing blob: SALES/NY/10000.json
Processing blob: SALES/NY/100000.json
Processing blob: SALES/NY/101000.json
Processing blob: SALES/NY/102000.json
Processing blob: SALES/NY/103000.json
Processing blob: SALES/NY/104000.json
Processing blob: SALES/NY/105000.json
Processing blob: SALES/NY/106000.json
Processing blob: SALES/NY/107000.json
Processing blob: SALES/NY/108000.json
Processing blob: SALES/NY/109000.json
Shape of SALES/NY/109000.json: (0, 43454)


KeyboardInterrupt: 

In [1]:
# Recreate the iterator blob_list
blob_list = bucket.list_blobs()

# Initialize a counter for the number of processed files
file_count = 0

# Flag to indicate if we have encountered the starting point
start_processing = False

# Iterate over each blob
for blob in blob_list:
    blob_name = blob.name
    print(f"Processing blob: {blob_name}")

    # Check if the blob is a file (not a directory) 
    if '.' in blob_name:
        # Check if the file is in the "Sales/NY" directory
        if blob_name.startswith("SALES/NY/") or blob_name == "SALES/NY":
            # Increment the file count
            file_count += 1

            # Check if we have reached the starting point
            if not start_processing:
                if file_count == 13:
                    start_processing = True
                else:
                    continue  # Skip until we reach the starting point

            # Check if we have processed 50 files
            if file_count > 50:
                print("Reached 50 files, stopping processing.")
                break  # Stop processing further files

            # Get the blob
            blob = bucket.blob(blob_name)

            # Download the blob content
            blob_content = blob.download_as_string()

            # Convert blob content to DataFrame
            df = pd.read_csv(StringIO(blob_content.decode('utf-8')))

            # Display the shape of the DataFrame
            print(f"Shape of {blob_name}: {df.shape}")

            # Store the DataFrame into Snowflake
            table_name = blob_name.split('/')[-1].split('.')[0]  # Extract table name from blob name
            df.to_sql(table_name, con=snowflake_engine, if_exists='append', index=False)
            print(f"DataFrame stored in Snowflake table: {table_name}")
        else:
            print("Blob is not in the 'Sales/NY' directory, skipping...")
    else:
        print("Blob is not a file, skipping...")

NameError: name 'bucket' is not defined